<center>
    <h1> CSC-343: ARTIFICIAL INTELLIGENCE </h1><br/>
    <h1> PROGRAMMING ASSIGNMENT 2 </h1><br/>
    <h1> FIRST-ORDER LOGIC (FOL) </h1><br/>
        
</center>

<hr/>

In [ ]:
from sent  import *
from utils import *
from kb    import *
from cnf   import *
from fol   import *

<hr/>

# 0. Documentation

# 0.1. First-Order Logic Knowledge Bases: `FolKB` (in `folkb.py`)


* The class `FolKB` can be used to represent a knowledge base of First-order logic sentences. 


* You would initialize and use it the same way as you would for `PropKB` <span style="color:red"><u>**except that the clauses are first-order definite clauses.**</u></span>

How to write such clauses to create a database and query them is given below: 

## 0.1.1. Criminal KB (from class/textbook)

In this section we create a `FolKB` based on the following paragraph, from class/textbook:<br/>

> <em>The law says that it is a crime for an American to sell weapons to hostile nations. The country Nono, an enemy of America, has some missiles, and all of its missiles were sold to it by Colonel West, who is American.</em><br/>


* The first step is to extract the facts and convert them into first-order definite clauses. 


* We'll store the clauses in list aptly named `clauses`.

In [ ]:
clauses = []

> <em>“... it is a crime for an American to sell weapons to hostile nations”</em><br/>

The keywords to look for here are 'crime', 'American', 'sell', 'weapon' and 'hostile'. 

We use predicate symbols to make meaning of them.

* `Criminal(x)`: `x` is a criminal
* `American(x)`: `x` is an American
* `Sells(x ,y, z)`: `x` sells `y` to `z`
* `Weapon(x)`: `x` is a weapon
* `Hostile(x)`: `x` is a hostile nation

Let us now combine them with appropriate variable naming to depict the meaning of the sentence. 

The criminal `x` is also the American `x` who sells weapon `y` to `z`, which is a hostile nation:


$$ \text{American}(x) \land \text{Weapon}(y) \land \text{Sells}(x, y, z) \land \text{Hostile}(z) \implies \text{Criminal} (x)$$

In [ ]:
clauses.append(sent("(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)"))

> <em>"The country Nono, an enemy of America"</em><br/>

We now know that Nono is an enemy of America. We represent these nations using the constant symbols `Nono` and `America`. the enemy relation is show using the predicate symbol `Enemy`.

$$\text{Enemy}(\text{Nono}, \text{America})$$

In [ ]:
clauses.append(sent("Enemy(Nono, America)"))

<em>"Nono ... has some missiles"</em><br/>

This states the existence of some missile which is owned by Nono. $\exists x \text{Owns}(\text{Nono}, x) \land \text{Missile}(x)$. 

We invoke existential instantiation to introduce a new constant `M1` which is the missile owned by Nono.

$$\text{Owns}(\text{Nono}, \text{M1}), \text{Missile}(\text{M1})$$

In [ ]:
clauses.append(sent("Owns(Nono, M1)"))
clauses.append(sent("Missile(M1)"))

<em>"All of its missiles were sold to it by Colonel West"</em><br/>

If Nono owns something and it classifies as a missile, then it was sold to Nono by West.

$$\text{Missile}(x) \land \text{Owns}(\text{Nono}, x) \implies \text{Sells}(\text{West}, x, \text{Nono})$$

In [ ]:
clauses.append(sent("(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)"))

<em>"West, who is American"</em><br/>

West is an American.

$$\text{American}(\text{West})$$

In [ ]:
clauses.append(sent("American(West)"))

We also know, from our understanding of language, that missiles are weapons and that an enemy of America counts as “hostile”.

$$\text{Missile}(x) \implies \text{Weapon}(x), \text{Enemy}(x, \text{America}) \implies \text{Hostile}(x)$$

In [ ]:
clauses.append(sent("Missile(x) ==> Weapon(x)"))

clauses.append(sent("Enemy(x, America) ==> Hostile(x)"))

Now that we have converted the information into first-order definite clauses we can create our first-order logic knowledge base.

In [ ]:
crime_kb = FolKB(clauses)

In [ ]:
crime_kb.clauses

<hr/>

# 0.2. Substitution

* The **`subst`** helper function substitutes variables with given values in first-order logic statements.

<img width="80%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/substitution.png">

Here's an example of how **`subst`** can be used:

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/substitution1.png">

In [ ]:
#Old Sentence with variables
s = sent('Student(x)')

#theta: dictionary mapping variables to terms 
theta = {s.args[0]: Symbol('Alice')}

new_sentence_w_terms = subst(theta, s)

print("Before substitution: ", s,     end='\n\n')
print("Theta:",                theta, end='\n\n')
print("After  substitution: ", new_sentence_w_terms)

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/substitution2.png">

In [ ]:
#Old Sentence with variables
s = sent('Student(x) & Takes(x, y)')
 
theta = {Symbol('x'): Symbol('Alice'), Symbol('y'): Symbol('CSC343')}

new_sentence_w_terms = subst(theta, s)

print("Before substitution: ", s,     end='\n\n')
print("Theta:",                theta, end='\n\n')
print("After  substitution: ", new_sentence_w_terms)

# 0.3.  Unification

* We sometimes require finding substitutions that make different logical sentences look identical. 


* This process, called **unification**, is done by the `unify` function.


* It takes as input two sentences and returns a <em>unifier</em> for them if one exists. 


* A unifier is a dictionary which stores the substitutions required to make the two sentences identical. 


* It does so by recursively unifying the components of a sentence, where the unification of a variable symbol `var` with a constant symbol `Const` is the mapping `{var: Const}`.

<img width="80%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/unification.png">

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/unification1.png">

In [ ]:
s1 = sent('Knows(Alice, FOL)')
s2 = sent('Knows(x, FOL)')
theta = unify(s1, s2)

print(theta)

print(subst(s1, theta) == subst(s2, theta))

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/unification2.png">

In [ ]:
s1 = sent('Knows(Alice, y)')
s2 = sent('Knows(x, z)')

theta = unify(s1, s2)

print(theta)

print(subst(s1, theta) == subst(s2, theta))

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/unification3.png">

In [ ]:
s1 = sent('Knows(Alice, y)')
s2 = sent('Knows(x, F(x))')

theta = unify(s1, s2)

print(theta)

print(subst(s1, theta) == subst(s2, theta))

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/unification4.png">

In [ ]:
s1 = sent('Knows(Alice, y)')
s2 = sent('Knows(Bob, y)')
theta = unify(s1, s2)

print(theta)

# 0.4 Standardizing Variables

<img width="80%" src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/standardizing.png">

In [ ]:
s1 = sent("Knows(Alice, x)")
s2 = sent("Knows(x, FOL)")

theta = unify(s1, s2)
print("Before standardizing, unification fails. Theta: ", theta, end="\n\n")

s3 = standardize_variables(s2)

theta = unify(s1, s3)
print("After standardizing `", s2,"`, new sentence: `", s3, "`", end="\n\n", sep="")
print("After standardizing, unification succeeds. Theta: ", theta)

# Q. Implement the Forward Chaining Inference Algorithm 

Basic steps of the Forward Chaining algorithm: 

1. We look at each rule in the knowledge base and see if the premises can be satisfied.


2. This is done by finding a substitution which unifies each of the premise with a clause in the `KB`. 


3. If we are able to unify the premises, the conclusion (with the corresponding substitution) is added to the `KB`. 


4. This inferencing process is repeated until either the query can be answered or till no new sentences can be added.


5. We test if the newly added clause unifies with the query in which case the substitution yielded by `unify` is an answer to the query. If we run out of sentences to infer, this means the query was a failure.


<b><span style="color:red;">Please note that function `fol_fc_ask` does not simply return `True` or `False`</span>
 
<span style="color:red;">It returns all substitutions which validate the query</span></b>.

The pseudocode for the forward-chaining algorithm is given below: 

<img src="https://raw.githubusercontent.com/fahadsultan/csc343/main/assets/imgs/forward_chaining.png">

Let's find out all the hostile nations. Note that we only told the `KB` that Nono was an enemy of America, not that it was hostile.

In [ ]:
kb_consts = list({c for clause in KB.clauses for c in constant_symbols(clause)})
def enum_subst(p):
    query_vars = list({v for clause in p for v in variables(clause)})
    for assignment_list in itertools.product(kb_consts, repeat=len(query_vars)):
        theta = {x: y for x, y in zip(query_vars, assignment_list)}
        yield theta

In [ ]:
def fol_fc_ask(KB, alpha):


In [ ]:
crime_kb = FolKB(map(sent, ['(American(x) & Weapon(y) & Sells(x, y, z) & Hostile(z)) ==> Criminal(x)',
                            'Owns(Nono, M1)',
                            'Missile(M1)',
                            '(Missile(x) & Owns(Nono, x)) ==> Sells(West, x, Nono)',
                            'Missile(x) ==> Weapon(x)',
                            'Enemy(x, America) ==> Hostile(x)',
                            'American(West)',
                            'Enemy(Nono, America)']))

answer = fol_fc_ask(crime_kb, sent('Hostile(x)'))

print(list(answer)) # The answer should be [{x: Nono}]

answer = fol_fc_ask(crime_kb, sent('Criminal(x)'))

print(list(answer)) # The answer should be [{x: West}]  

<hr />



# _Documentation from previous labs:_

## 0.1. `sent.py`

You are given a python file `sent.py`

This file contains a class `Sent`

 **`Sent`**: A PL sentence with an operator `op` and 0 or more arguments `args`.\
    `op` is a str like `'+'` or `'sin'`;\
    `args` are Sentences.\
    \
    `Sent('x')` or `Symbol('x')` creates a symbol (a nullary Sent).\
    `Sent('-', x)` creates a unary sentence; \
    `Sent('+', x, 1)` creates a binary sentence.

**`Symbol`** is not an explicit type; it is any `Sent` with 0 `args`.

### Creating Symbols

`sent.py` comes with a function `Symbol(...)` that takes in as input name of the propositional symbol you want to create (string type; <u><i>please start names with uppercase letters</i></u>) and returns an object of type `Sent` with 0 `args` and `op` equal to the provided name. 

### Creating PL Sentences:

There are **three ways** of constructing sentences: 

1. Infix notation

2. `Sent` constructor

3. `sent(...)` method

The first _two_ methods are shown in the table below, in the third and fifth columns respectively.

Here is a table of the operators that can be used to form sentences. 

<!-- Note that we have a problem: we want to use Python operators to make sentences, so that our programs (and our interactive sessions like the one here) will show simple code. But Python does not allow implication arrows as operators, so for now we have to use a more verbose notation that Python does allow: `|'==>'|` instead of just `==>`. Alternately, you can always use the more verbose `Sent` constructor forms:
 -->
| Operation                | Textbook<br/>/Slides | Python<br/> INFIX<br/> NOTATION | Python<br/>Output | Python<br/>`Sent`<br/>CONSTRUCTOR
|--------------------------|----------------------|-------------------------|---|---|
| Negation                 | &not; P      | `~P`                       | `~P` | `Sent('~', P)`
| And                      | P &and; Q       | `P & Q`                     | `P & Q` | `Sent('&', P, Q)`
| Or                       | P &or; Q | `P`<tt> &#124; </tt>`Q`| `P`<tt> &#124; </tt>`Q` | `Sent('`&#124;`', P, Q)`
| Inequality (Xor)         | P &ne; Q     | `P ^ Q`                | `P ^ Q`  | `Sent('^', P, Q)`
| Implication                  | P &rarr; Q    | `P` <tt>&#124;</tt>`'==>'`<tt>&#124;</tt> `Q`   | `P ==> Q` | `Sent('==>', P, Q)`
| Reverse Implication      | Q &larr; P     | `Q` <tt>&#124;</tt>`'<=='`<tt>&#124;</tt> `P`  |`Q <== P` | `Sent('<==', Q, P)`
| Equivalence            | P &harr; Q   | `P` <tt>&#124;</tt>`'<=>'`<tt>&#124;</tt> `Q`   |`P <=> Q` | `Sent('<=>', P, Q)`

Note that there are two operators we haven't covered in class:  Reverse Implication `<==` and Inequality (aka Xor) `^`.

**You are to implement these yourself**; the truth table for them is as follows: 

    
| P | Q | | P `<==` Q | | P `^` Q |
|--------------------------|-|----------------------|-|-------------------------|---|
| T | T | | T | | F | 
| T | F | | T | | T | 
| F | T | | F | | T | 
| F | F | | T | | F |  

**`sent(..)`** function provides a third way of making PL sentences here. 

The `sent` function takes in one input: 
* a string representing a PL sentence in the infix notation 

and returns 
* a `Sent` object for that sentence. For example, 

In [ ]:
from sent  import *
from utils import *
from kb    import *
from cnf   import *

p = Symbol('P')
print("0. Creating a propositional symbol P:\t\t", p, "\n")
print("Type of prop symbol P:\t\t", type(p))

p = Symbol('P')
print("Arguments of prop symbol P:\t", p.args)
print("Operator  of prop symbol P:\t", p.op)

print()

q = Symbol('Q') 
# P OR Q 

s = Sent('|', p, q)
print("1. Creating a sentence s, using Sent:\t\t", s, "\n")
print("Type       of sentence `s`:\t", type(s))
print("Argument 1 of sentence `s`:\t", s.args[0])
print("Argument 2 of sentence `s`:\t", s.args[1])
print("Operator   of sentence `s`:\t", s.op)

print()

s = p | q
print("2. Creating a sentence s, using infix notation:\t", s, "\n")
print("Type       of sentence `s`:\t", type(s))
print("Argument 1 of sentence `s`:\t", s.args[0])
print("Argument 2 of sentence `s`:\t", s.args[1])
print("Operator   of sentence `s`:\t", s.op)

print()

s = sent('~(P & Q)  ==>  (~P | ~Q)')
print("3. Creating the sentence s, using sent method:\t", s, "\n")
print("Type       of sentence `s`:\t", type(s))
print("Argument 1 of sentence `s`:\t", s.args[0])
print("Argument 2 of sentence `s`:\t", s.args[1])
print("Operator   of sentence `s`:\t", s.op)

## 0.2. `utils.py`

The other file you are given is `utils.py` which contains the following functions you might need in your implementation of `pl_true` and `tt_entails`: 
    
* **`extend`** takes three inputs: i. `dict` (dictionary) object ii. `key` iii. `value` and returns an _extended_ dictionary that has all key-value pairs from the input dictionary AND the new `key:value` pair. The extended dictionary that is returned is a new object and leaves the input dictionary unchanged. 


* **`is_prop_symbol`** takes as input an operator of a sentence and returns True if it is a propositional symbol and False otherwise. 


* **`prop_symbols`** takes as input a sentence and returns a list of all the propositional symbols in it. 

## 0.3. `kb.py`

A PL knowledge base to which you can `tell` and `ask_if_true` sentences.

## 0.4. `cnf.py`

Contains a set of functions: `to_cnf`, `eliminate_implications`, `move_not_inwards` and `distribute_and_over_or` for converting any sentence to CNF form. 


<hr/>